# Modules Import

In [1]:
from qiskit import Aer
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
molecule = Molecule(
    geometry=[
        ["H", [0.0, 0.0, 0.0]],
        # ["Li", [0.0, 0.0, 0.0]],
        # ["Be", [0.0, 0.0, 0.345]],
        ["H", [0.0, 0.0, 0.734]]], 
    charge=0, 
    multiplicity=1
)
driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)

es_problem = ElectronicStructureProblem(driver)
qubit_converter = QubitConverter(JordanWignerMapper())

In [3]:
from qiskit.providers.aer import StatevectorSimulator
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import VQEUCCFactory

quantum_instance = QuantumInstance(backend=Aer.get_backend("aer_simulator_statevector"))
vqe_solver = VQEUCCFactory(quantum_instance)

In [4]:
from qiskit.algorithms import NumPyMinimumEigensolver
numpy_solver = NumPyMinimumEigensolver()

In [5]:
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal

tl_circuit = TwoLocal(
    rotation_blocks=["h", "rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=2,
    parameter_prefix="y",
)

another_solver = VQE(
    ansatz=tl_circuit,
    quantum_instance=QuantumInstance(Aer.get_backend("aer_simulator_statevector")),
)

# Classical Calculation (numpy_solver)

In [6]:
from qiskit_nature.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(qubit_converter, numpy_solver)
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.858255290955
  - computed part:      -1.858255290955
~ Nuclear repulsion energy (Hartree): 0.720949878638
> Total ground state energy (Hartree): -1.137305412318
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.38705898]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.38705898]
    - computed part:      [0.0  0.0  1.38705898]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.00000001]  Total: 0.00000001
 


# IBM Calculation (vqe_solver)

In [7]:
calc = GroundStateEigensolver(qubit_converter, vqe_solver)
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.858255290898
  - computed part:      -1.858255290898
~ Nuclear repulsion energy (Hartree): 0.720949878638
> Total ground state energy (Hartree): -1.137305412261
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.38705898]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.3870588]
    - computed part:      [0.0  0.0  1.3870588]
  > Dipole moment (a.u.): [0.0  0.0  0.00000018]  Total: 0.00000018
                 (debye): [0.0  0.0  0.00000045]  Total: 0.00000045
 


In [8]:
dir(res)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_aux_operator_eigenvalues',
 '_computed_dipole_moment',
 '_computed_energies',
 '_eigenenergies',
 '_eigenstates',
 '_extracted_transformer_dipoles',
 '_extracted_transformer_energies',
 '_hartree_fock_energy',
 '_magnetization',
 '_nuclear_dipole_moment',
 '_nuclear_repulsion_energy',
 '_num_particles',
 '_raw_result',
 '_reverse_dipole_sign',
 '_total_angular_momentum',
 'aux_operator_eigenvalues',
 'combine',
 'computed_dipole_moment',
 'computed_energies',
 'dipole_moment',
 'dipole_moment_in_debye',
 'eigenenergies',
 'eigenstates',
 'electronic_dipole_moment',
 'e

In [9]:
print(res.total_energies)

[-1.13730541]
